In [152]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

#kaggle.com/code/kritanjalijain/apriori-algorithm-hotel-management-system

In [153]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [174]:
# load data:
df_ht = pd.read_csv("hotel_bookings.csv")

df_ht.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [175]:
# Replace missing values:
# agent: If no agency is given, booking was most likely made without one.
# company: If none given, it was most likely private.
# rest schould be self-explanatory.


nan_replacements = {"children:": 0.0,"country": "Unknown", "agent": 0, "company": 0}
df_clean = df_ht.fillna(nan_replacements)

# "meal" contains values "Undefined", which is equal to SC.
df_clean["meal"].replace("Undefined", "SC", inplace=True)

# Some rows contain entreis with 0 adults, 0 children and 0 babies. 
# I'm dropping these entries with no guests.
zero_guests = list(df_clean.loc[df_clean["adults"]
                   + df_clean["children"]
                   + df_clean["babies"]==0].index)

df_clean.drop(df_clean.index[zero_guests], inplace=True)


#
df_clean = df_clean[df_clean['is_canceled'] == 0]

df_clean.head(5)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,0.0,0.0,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,0.0,0.0,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,0.0,0.0,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,0.0,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,0.0,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [176]:
# adr 0 이하 이상값으로 판단하여 제거
df_clean = df_clean[df_clean['adr'] > 0]

# adr 구간화
labels = ['low','middle','high']
df_clean['adr_bin'] = pd.qcut(df_clean['adr'],3,labels=labels)
df_clean.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,adr_bin
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,0.0,0.0,0,Transient,75.0,0,0,Check-Out,2015-07-02,low
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,304.0,0.0,0,Transient,75.0,0,0,Check-Out,2015-07-02,low
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,240.0,0.0,0,Transient,98.0,0,1,Check-Out,2015-07-03,middle
5,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,240.0,0.0,0,Transient,98.0,0,1,Check-Out,2015-07-03,middle
6,Resort Hotel,0,0,2015,July,27,1,0,2,2,...,0.0,0.0,0,Transient,107.0,0,0,Check-Out,2015-07-03,middle


In [177]:
# portugal_df = df_clean[df_clean["country"] == 'PRT']
# france_df = df_clean[df_clean["country"] == 'FRA']
# england_df = df_clean[df_clean["country"] == 'GBR']
# germany_df = df_clean[df_clean["country"] == 'DEU']
# italy_df = df_clean[df_clean["country"] == 'ITA']

df_clean = pd.get_dummies(data=df_clean,
               columns=['hotel','meal','market_segment','distribution_channel','country','adr_bin','arrival_date_year','customer_type'],
               drop_first=False,
               dtype=None
               )
df_clean.columns.values

array(['is_canceled', 'lead_time', 'arrival_date_month',
       'arrival_date_week_number', 'arrival_date_day_of_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults',
       'children', 'babies', 'is_repeated_guest',
       'previous_cancellations', 'previous_bookings_not_canceled',
       'reserved_room_type', 'assigned_room_type', 'booking_changes',
       'deposit_type', 'agent', 'company', 'days_in_waiting_list', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date',
       'hotel_City Hotel', 'hotel_Resort Hotel', 'meal_BB', 'meal_FB',
       'meal_HB', 'meal_SC', 'market_segment_Aviation',
       'market_segment_Complementary', 'market_segment_Corporate',
       'market_segment_Direct', 'market_segment_Groups',
       'market_segment_Offline TA/TO', 'market_segment_Online TA',
       'distribution_channel_Corporate', 'distribution_channel_Direct',
       'distribution_channel_GDS', 'd

In [178]:
clean_france_df = df_clean[[ 
        'agent',
        # 'company',
       'children', 
       'babies', 
       'previous_cancellations',
       'previous_bookings_not_canceled',  
       'booking_changes', 
       'required_car_parking_spaces', 
       'total_of_special_requests',
       'hotel_City Hotel', 'hotel_Resort Hotel',
       'country_FRA','country_PRT','country_GBR','country_ESP','country_DEU',
       'meal_BB', 'meal_FB','meal_HB', 'meal_SC',
        'market_segment_Aviation','market_segment_Complementary', 
        # 'market_segment_Corporate',
    #    'market_segment_Direct',
       'market_segment_Groups',
    #    'market_segment_Offline TA/TO', 'market_segment_Online TA',
       'distribution_channel_Corporate', 'distribution_channel_Direct',
       'distribution_channel_GDS',
    #    , 'distribution_channel_TA/TO',
       'adr_bin_low','adr_bin_middle','adr_bin_high',
    #    'arrival_date_year_2015','arrival_date_year_2016',
        'customer_type_Group',
        'customer_type_Transient',
        'customer_type_Contract',
        'customer_type_Transient-Party',
       ]]
#df_clean.dropna(axis=0, subset =)
clean_france_df.dropna()

,agent,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,required_car_parking_spaces,total_of_special_requests,hotel_City Hotel,hotel_Resort Hotel,...,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,adr_bin_low,adr_bin_middle,adr_bin_high,customer_type_Group,customer_type_Transient,customer_type_Contract,customer_type_Transient-Party
2,0.0,0.0,0,0,0,0,0,0,0,1,...,0,1,0,1,0,0,0,1,0,0
3,304.0,0.0,0,0,0,0,0,0,0,1,...,1,0,0,1,0,0,0,1,0,0
4,240.0,0.0,0,0,0,0,0,1,0,1,...,0,0,0,0,1,0,0,1,0,0
5,240.0,0.0,0,0,0,0,0,1,0,1,...,0,0,0,0,1,0,0,1,0,0
6,0.0,0.0,0,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,394.0,0.0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0
119386,9.0,0.0,0,0,0,0,0,2,1,0,...,0,0,0,0,0,1,0,1,0,0
119387,9.0,0.0,0,0,0,0,0,4,1,0,...,0,0,0,0,0,1,0,1,0,0
119388,89.0,0.0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0


In [179]:
# Defining the hot encoding function to make the data suitable
# for the concerned libraries
def hot_encode(x):
    if(x<= 0):
        return 0   
    if(x>= 1):
        return 1

# Encoding the datasets
clean_france_df_encoded = clean_france_df.applymap(hot_encode)
clean_france_df = clean_france_df_encoded.copy()

In [180]:
# Building the model
frq_items = apriori(clean_france_df, min_support = 0.05, use_colnames = True)

# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
pd.set_option('display.max_row', 500)
rules.head(500)


#
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequent_len"] = rules["consequents"].apply(lambda x: len(x))


In [187]:
rules_filtered1 = rules[
        (rules['antecedent_len'] == 1) &
        (rules['consequent_len'] == 1) &
        (rules['support'] > 0.05) &
        (rules['confidence'] > 0.5) &
        (rules['lift'] > 1.1)  
    ]

rules_filtered1.head(500)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
100,(distribution_channel_Corporate),(meal_BB),0.068608,0.769295,0.066060,0.962860,1.251614,0.013280,6.211771,1,1
12,(country_DEU),(agent),0.082139,0.843558,0.078337,0.953716,1.130588,0.009048,3.380055,1,1
117,(market_segment_Groups),(customer_type_Transient-Party),0.101979,0.249591,0.096692,0.948156,3.798836,0.071239,14.474380,1,1
17,(meal_SC),(agent),0.099866,0.843558,0.093367,0.934916,1.108302,0.009124,2.403704,1,1
6,(country_FRA),(agent),0.114828,0.843558,0.107184,0.933428,1.106538,0.010320,2.349983,1,1
29,(children),(customer_type_Transient),0.073309,0.705551,0.064820,0.884201,1.253205,0.013097,2.542751,1,1
65,(meal_SC),(hotel_City Hotel),0.099866,0.614801,0.087998,0.881157,1.433240,0.026600,3.241246,1,1
47,(required_car_parking_spaces),(customer_type_Transient),0.099526,0.705551,0.086009,0.864184,1.224835,0.015788,2.167997,1,1
105,(adr_bin_low),(meal_BB),0.333338,0.769295,0.285142,0.855414,1.111946,0.028707,1.595630,1,1
58,(country_FRA),(hotel_City Hotel),0.114828,0.614801,0.095942,0.835529,1.359023,0.025346,2.342045,1,1


In [182]:
rules_filtered1 = rules[
        (rules['antecedent_len'] == 1) &
        (rules['consequent_len'] == 2) &
        (rules['support'] > 0.05) &
        (rules['confidence'] > 0.5) &
        (rules['lift'] > 1.15)  
    ]

rules_filtered1.head(500)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
245,(meal_SC),"(hotel_City Hotel, agent)",0.099866,0.546901,0.085218,0.853322,1.560286,0.030601,3.089079,1,2
579,(meal_SC),"(customer_type_Transient, hotel_City Hotel)",0.099866,0.428980,0.080585,0.806931,1.881046,0.037745,2.957600,1,2
355,(meal_SC),"(customer_type_Transient, agent)",0.099866,0.597768,0.080517,0.806249,1.348766,0.020820,2.076027,1,2
139,(children),"(customer_type_Transient, agent)",0.073309,0.597768,0.058484,0.797770,1.334580,0.014662,1.988978,1,2
233,(country_DEU),"(hotel_City Hotel, agent)",0.082139,0.546901,0.065433,0.796616,1.456599,0.020511,2.227796,1,2
225,(country_FRA),"(hotel_City Hotel, agent)",0.114828,0.546901,0.090519,0.788300,1.441392,0.027719,2.140281,1,2
377,(adr_bin_high),"(customer_type_Transient, agent)",0.333338,0.597768,0.243991,0.731963,1.224493,0.044732,1.500656,1,2
133,(children),"(agent, meal_BB)",0.073309,0.629626,0.053592,0.731041,1.161071,0.007435,1.377064,1,2
293,(country_FRA),"(meal_BB, agent)",0.114828,0.629626,0.083501,0.727186,1.154950,0.011203,1.357608,1,2
385,(children),"(customer_type_Transient, meal_BB)",0.073309,0.557816,0.052924,0.721933,1.294214,0.012031,1.590207,1,2


In [186]:
rules_filtered2 = rules[ 
      (rules['antecedent_len'] == 2) &
    #   (rules['consequent_len'] == 2) &
      (rules['support'] > 0.05) &
       (rules['confidence'] > 0.5) &
       (rules['lift'] > 2)  
    ]

rules_filtered2.head(500)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
357,"(agent, market_segment_Groups)",(customer_type_Transient-Party),0.073827,0.249591,0.071633,0.970284,3.887494,0.053206,25.252887,2,1
581,"(hotel_City Hotel, market_segment_Groups)",(customer_type_Transient-Party),0.057516,0.249591,0.055036,0.956882,3.833798,0.040681,17.403711,2,1
707,"(meal_BB, market_segment_Groups)",(customer_type_Transient-Party),0.070379,0.249591,0.066278,0.941723,3.773062,0.048712,12.876616,2,1
734,"(adr_bin_low, market_segment_Groups)",(customer_type_Transient-Party),0.059724,0.249591,0.056194,0.940908,3.769796,0.041288,12.699003,2,1
1292,"(total_of_special_requests, meal_SC)","(customer_type_Transient, hotel_City Hotel)",0.059887,0.428980,0.052733,0.880546,2.052650,0.027043,4.780251,2,2
626,"(adr_bin_low, country_GBR)",(hotel_Resort Hotel),0.058184,0.385199,0.050744,0.872131,2.264104,0.028332,4.808057,2,1
1652,"(total_of_special_requests, meal_SC)","(customer_type_Transient, hotel_City Hotel, ag...",0.059887,0.380198,0.052052,0.869170,2.286095,0.029283,4.737440,2,3
1243,"(required_car_parking_spaces, meal_BB)","(customer_type_Transient, hotel_Resort Hotel)",0.082043,0.276571,0.052366,0.638266,2.307783,0.029675,1.999892,2,2
1650,"(customer_type_Transient, meal_SC)","(total_of_special_requests, hotel_City Hotel, ...",0.083352,0.307162,0.052052,0.624489,2.033094,0.026450,1.845055,2,3
1240,"(customer_type_Transient, required_car_parking...","(meal_BB, hotel_Resort Hotel)",0.086009,0.295334,0.052366,0.608840,2.061529,0.026964,1.801478,2,2


# ADR관련

In [135]:
rules_filtered2 = rules[ 
      (rules['antecedent_len'] == 2) &
      (rules['support'] > 0.05) &
       (rules['confidence'] > 0.5) &
       (rules['lift'] > 1.2)  
    ]

rules_filtered2.head(500)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
254,"(distribution_channel_Corporate, company)",(meal_BB),0.054845,0.769295,0.053278,0.971429,1.262752,0.011086,8.074685,2,1
287,"(total_of_special_requests, meal_SC)",(hotel_City Hotel),0.059887,0.614801,0.058061,0.969511,1.576951,0.021243,12.633959,2,1
434,"(total_of_special_requests, meal_SC)","(hotel_City Hotel, agent)",0.059887,0.546901,0.057230,0.955631,1.747356,0.024478,10.212145,2,2
188,"(agent, meal_SC)",(hotel_City Hotel),0.093367,0.614801,0.085218,0.912726,1.484589,0.027816,4.413687,2,1
366,"(adr_bin_low, country_GBR)",(hotel_Resort Hotel),0.058184,0.385199,0.050744,0.872131,2.264104,0.028332,4.808057,2,1
327,"(country_FRA, meal_BB)",(hotel_City Hotel),0.090137,0.614801,0.077956,0.864853,1.406720,0.022539,2.850216,2,1
277,"(total_of_special_requests, country_FRA)",(hotel_City Hotel),0.063839,0.614801,0.053973,0.845464,1.375184,0.014725,2.492622,2,1
169,"(country_FRA, agent)",(hotel_City Hotel),0.107184,0.614801,0.090519,0.844521,1.373649,0.024622,2.477496,2,1
176,"(country_DEU, agent)",(hotel_City Hotel),0.078337,0.614801,0.065433,0.835276,1.358612,0.017271,2.338448,2,1
532,"(country_FRA, meal_BB)","(hotel_City Hotel, agent)",0.090137,0.546901,0.073227,0.812396,1.485453,0.023931,2.415187,2,2


In [136]:
# for i in range(0,len(dataset)):
#     transactions.append({str(dataset.values[i,j]) for j in range(0,len(dataset.columns)) if str(dataset.values[i,j]) != 'nan'})

# print('Transactions:\n')
# for transaction in transactions:
#     print(transaction)

1. customers travel and stay with children have more special requests
2. customers who travel and stay during weekdays have more special requests

In [137]:
# def init_candidates(transactions):
#     candidates = dict()
#     for transaction in transactions:
#         for item in transaction:
#             itemset = set()
#             itemset.add(item)
#             itemset = frozenset(itemset)
#             if itemset not in candidates:
#                 candidates[itemset] = 1
#             else:
#                 candidates[itemset] += 1
                
#     return candidates


# def prune_candidates(candidates,support):
#     fp = dict()
#     for key in candidates:
#         if candidates[key] >= support:
#             fp[key] = candidates[key]
    
#     return fp


# def get_count(transactions,itemset):
#     itemset = set(itemset)
#     count = 0
#     for transaction in transactions:
#         if itemset.issubset(transaction):
#             count += 1
            
#     return count


# def get_candidates(fp,transactions,fp_length):
#     candidates = dict()
#     for key1 in fp:
#         for key2 in fp:
#             if key2 != key1:
#                 itemset1 = set(key1)
#                 itemset2 = set(key2)
#                 itemset = itemset1.union(itemset2)
#                 itemset = frozenset(itemset)
#                 if itemset not in candidates and len(itemset) == fp_length:
#                     candidates[itemset] = get_count(transactions,itemset)
    
#     return candidates


# def get_itemset_length(candidates):
#     itemset_len = 0
#     for key in candidates:
#         itemset_len = len(key)
#         break
    
#     return itemset_len


# import itertools

# def get_rules(fp):
#     rules = []
#     for itemset in fp:
#         itemset = set(itemset)
#         for length in range(1,len(itemset)):
#             subsets = set(itertools.combinations(itemset, length))
#             for subset in subsets:
#                 lhs = set(subset)
#                 rhs = itemset.difference(lhs)
#                 rules.append([lhs,rhs])
    
#     return rules


# def prune_rules(rules,confidence,transactions):
#     fp = []
#     for rule in rules:
#         lhs = rule[0]
#         rhs = rule[1]
#         lhs_rhs = lhs.union(rhs)
#         lhs_count = get_count(transactions,lhs)
#         lhs_rhs_count = get_count(transactions,lhs_rhs)
#         conf = lhs_rhs_count/lhs_count
#         if conf >= confidence:
#             support = lhs_rhs_count/len(transactions)
#             fp.append([lhs,rhs,support,conf])
    
#     return fp

# def display(candidates,fp,length):
#     print()
#     print("****** {}-Frequent Candidates ******".format(length))
#     for key in candidates:
#         print(set(key),' : ',candidates[key])
#     print()
#     print("****** {}-Frequent Itemlist ******".format(length))
#     for key in fp:
#         print(set(key),' : ',fp[key])
# def apriori(transactions,support,confidence):
#     support = support/100 * len(transactions)
#     confidence = confidence/100
#     candidates = init_candidates(transactions)
#     old_fp = prune_candidates(candidates,support)
#     final_fp = dict()
#     display(candidates,old_fp,1)
    
#     while True:
#         new_fp_length = get_itemset_length(candidates) + 1
#         candidates = get_candidates(old_fp,transactions,new_fp_length)
#         new_fp = prune_candidates(candidates,support)
#         display(candidates,new_fp,new_fp_length)
#         if len(new_fp)<1:
#             final_fp = old_fp
#             break
#         else:
#             old_fp = new_fp
    
#     rules = get_rules(final_fp)
#     fp = prune_rules(rules,confidence,transactions)
#     return fp
# support = float(input('Enter support percentage : '))
# confidence = float(input('Enter confidence percentage : '))
# fp = apriori(transactions,support,confidence)